In [3]:
import requests
import pandas as pd
from io import StringIO
import datetime   
    
def crawl_dividend_policy(date):
    
    def exclude_redundant_words(datestr):
        if len(datestr) > 10:
            datestr = datestr[:10]

        datestr = pd.to_datetime(datestr)
        return datestr

    # 只是用日期的年份
    year = date.year
    
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36'}
    url = 'https://goodinfo.tw/StockInfo/StockDividendScheduleList.asp?MARKET_CAT=%E4%B8%8A%E5%B8%82&INDUSTRY_CAT=%E5%85%A8%E9%83%A8&YEAR=' + str(year)
    sess = requests.Session()
    
    try:
        r = sess.get(url, headers=headers)
        r.encoding = 'utf-8'
    except:
        print('**WARRN: requests cannot get ' + str(year) + ' dividend policy')
        return

    html_df = pd.read_html(StringIO(r.text))
    
    # 取第 5 個表格
    df = html_df[4]
    df = df.iloc[39:, 0:17]

    col_1 = df.columns.droplevel([1, 2])
    col_2 = df.columns.droplevel([0, 2])
    col_3 = df.columns.droplevel([0, 1])
    # 整理欄位名稱
    df.columns = df.columns.droplevel([1,2])
    df.rename(columns = {col_1[-7] : col_2[0] + col_3[0],
                         col_1[-6] : col_2[0] + col_3[1],
                         col_1[-5] : col_2[0] + col_3[2],
                         col_1[-4] : col_2[1] + col_3[0],
                         col_1[-3] : col_2[1] + col_3[1],
                         col_1[-2] : col_2[1] + col_3[2],
                         col_1[-1] : col_2[2]}, inplace=True)

    # 排除中間的欄位說明
    df = df[df['市場'] == '上市']
    df = df.reset_index(drop=True)
    
    # 開始處理股利合計
    dividend_total = df.copy()
    dividend_total['date'] = pd.to_datetime(str(year) + '-06-30')

    dividend_total = dividend_total.rename(columns={'代碼':'stock_id'})
    dividend_total = dividend_total.sort_values(['stock_id', 'date']).set_index(['stock_id', 'date'])

    dividend_total = dividend_total.iloc[:, -7:]
    dividend_total = dividend_total.apply(lambda s: pd.to_numeric(s, errors='corece'))
    dividend_total = dividend_total[dividend_total.columns[dividend_total.isnull().all() == False]]
    # 合計股利
    dividend_total = dividend_total.groupby(['stock_id', 'date']).agg('sum')

    # 取出重要的時間欄位, 排除贅字後, 整理為 pd.to_datetime
    date_columns = ['股東會日期', '除息交易日', '除權交易日', '現金股利發放日']
    df[date_columns] = df[date_columns].astype(str).applymap(exclude_redundant_words)

    # 開始處理除權息日程
    dividend_schedule = df.copy()
    # 將除息交易日設定為 date, 缺少除息交易日使用除權交易日代替
    dividend_schedule['date'] = dividend_schedule['除息交易日']
    dividend_schedule['date'].fillna(dividend_schedule['除權交易日'], inplace=True)

    dividend_schedule = dividend_schedule.rename(columns={'代碼':'stock_id'})
    dividend_schedule = dividend_schedule.sort_values(['stock_id', 'date']).set_index(['stock_id', 'date'])

    # 排除日期, 其餘轉為數字
    df_dates = dividend_schedule[date_columns]
    dividend_schedule = dividend_schedule.apply(lambda s: pd.to_numeric(s, errors='corece'))
    dividend_schedule = dividend_schedule[dividend_schedule.columns[dividend_schedule.isnull().all() == False]]
    dividend_schedule[date_columns] = df_dates
    dividend_schedule.columns = '股利政策之' + dividend_schedule.columns
    
    tbs = {'dividend_total' : dividend_total,
           'dividend_schedule' : dividend_schedule }
    
    return tbs
tbs = crawl_dividend_policy(datetime.date(2017, 1, 1))
import sqlite3
import os
from finlab.crawler import widget, date_range
conn = sqlite3.connect(os.path.join('data', "data.db"))
import pandas as pd
total  =tbs['dividend_total']
schedule = tbs['dividend_schedule']
total.to_sql("dividend_total" ,conn,if_exists='append',index=True)
schedule.to_sql("dividend_schedule " , conn , if_exists='append' , index=True)